In [1]:
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers, models, layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import ResNet50V2

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import io
from PIL import Image

In [2]:
from minio import Minio
from minio.error import ResponseError

minioClient = Minio(
                'minio-service.kubeflow:9000',
                access_key='minio', 
                secret_key='minio123', 
                secure=False
            )

In [3]:
INIT_LR = 1e-05  #1e-05  # This value is specific to what model is chosen: Inception, VGG or ResnNet.
EPOCHS = 30 
BS = 8

print("Loading images...")

data = []
labels = []

Loading images...


In [4]:

# read all X-Rays in the specified path, and resize them all to 256x256

for i in minioClient.list_objects('dataset', prefix='covid19', recursive=True):
    label = i.object_name.split(os.path.sep)[-2]

    minioObj = minioClient.get_object('dataset', i.object_name)
    byteArray = minioObj.read()
    pil_image = Image.open(io.BytesIO(byteArray)).convert('RGB')
    image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (256, 256))
    data.append(image)
    labels.append(label)

    
#normalise pixel values to real numbers between 0.0 - 1.0 
data = np.array(data) / 255.0
labels = np.array(labels)

# perform one-hot encoding for a 3-class labeling 
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels)
labels = to_categorical(integer_encoded)

print("... ... ", len(data), "images loaded in 3x classes:")
print(label_encoder.classes_)

... ...  227 images loaded in 3x classes:
['covid' 'normal' 'pneumonia_bac']


In [5]:
(x_train, x_val, y_train, y_val) = train_test_split(data, labels, test_size=0.20, stratify=labels)

In [6]:
# Katib 파라미터
LEARNING_RATE = 0.00001 # List: 0.001, 0.0001, 0.0003, 0.00001, 0.00003
DENSE = 128             # Range: 50-200

In [7]:
model = Sequential()
adam_s = Adam(learning_rate = LEARNING_RATE)

#model.add(VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet', pooling='average'))
model.add(ResNet50V2(input_shape=(256, 256, 3),include_top=False, weights='imagenet',pooling='average'))

for layer in model.layers:
    layer.trainable = False

model.add(BatchNormalization())
model.add(Flatten())
model.add(Dense(DENSE, activation='relu'))
model.add(Dense(DENSE, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=adam_s, metrics=['accuracy'])


model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50v2 (Model)           (None, 8, 8, 2048)        23564800  
_________________________________________________________________
batch_normalization (BatchNo (None, 8, 8, 2048)        8192      
_________________________________________________________________
flatten (Flatten)            (None, 131072)            0         
_________________________________________________________________
dense (Dense)                (None, 128)               16777344  
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 40,367,235
Trainable params: 16,798,339
Non-trainable params: 23,568,896
___________________________________

In [8]:
# train the head of the network
print("Training the full stack model...")
H = model.fit(x_train, y_train, epochs=10, validation_data=(x_val, y_val), batch_size=8)

Training the full stack model...
Train on 181 samples, validate on 46 samples
Epoch 1/10
181/181 [==============================] - 25s 136ms/sample - loss: 0.9162 - accuracy: 0.5856 - val_loss: 1.0557 - val_accuracy: 0.3913
Epoch 2/10
181/181 [==============================] - 19s 104ms/sample - loss: 0.3127 - accuracy: 0.9392 - val_loss: 0.8635 - val_accuracy: 0.5652
Epoch 3/10
181/181 [==============================] - 18s 102ms/sample - loss: 0.1342 - accuracy: 0.9834 - val_loss: 0.7246 - val_accuracy: 0.6739
Epoch 4/10
181/181 [==============================] - 20s 108ms/sample - loss: 0.0836 - accuracy: 0.9834 - val_loss: 0.6709 - val_accuracy: 0.6957
Epoch 5/10
181/181 [==============================] - 20s 108ms/sample - loss: 0.0484 - accuracy: 0.9945 - val_loss: 0.6313 - val_accuracy: 0.7174
Epoch 6/10
181/181 [==============================] - 20s 108ms/sample - loss: 0.0375 - accuracy: 1.0000 - val_loss: 0.6158 - val_accuracy: 0.6957
Epoch 7/10
181/181 [====================

In [9]:
os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minio',
    'AWS_SECRET_ACCESS_KEY': 'minio123',
    'S3_USE_HTTPS'         : '0',   # Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0'    # If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})  

model.save("s3://model/new-covid/1")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: s3://model/new-covid/1/assets


# Serving Test

In [10]:
print("[INFO] loading untrained test images...")
imagePathTest = "./all/test"
imagePathsTest = list(paths.list_images(imagePathTest))
print(len(imagePathsTest))

dataTest = []
labelsTest = []

[INFO] loading untrained test images...
0


In [11]:
for i in minioClient.list_objects('dataset', prefix='covid19', recursive=True):
    labelTest = i.object_name.split(os.path.sep)[-2]

    minioObj = minioClient.get_object('dataset', i.object_name)
    byteArray = minioObj.read()
    pil_image = Image.open(io.BytesIO(byteArray)).convert('RGB')
    image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
    imageTest = cv2.resize(image, (256, 256))
    dataTest.append(imageTest)
    labelsTest.append(labelTest)

In [12]:
# convert the data and labels to NumPy arrays while scaling the pixel
# intensities to the range [0, 255]
dataTest = np.array(dataTest) / 255.0
labelsTest = np.array(labelsTest)


# perform one-hot encoding on the labels
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labelsTest)
labelsTest = to_categorical(integer_encoded)

print(labelsTest)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1.

In [13]:
predTest = model.predict(dataTest, batch_size=BS)
print(predTest)

[[4.53233153e-01 4.13356572e-01 1.33410186e-01]
 [4.97456789e-01 1.21640563e-01 3.80902618e-01]
 [2.30277285e-01 2.35954970e-01 5.33767700e-01]
 [8.19995224e-01 1.29434839e-01 5.05699068e-02]
 [5.57791233e-01 1.85351551e-01 2.56857216e-01]
 [5.84322870e-01 3.15349907e-01 1.00327276e-01]
 [7.99425006e-01 1.25826746e-01 7.47482106e-02]
 [9.05752122e-01 4.52578552e-02 4.89900075e-02]
 [8.59755337e-01 4.93400469e-02 9.09046605e-02]
 [1.21257819e-01 5.02093554e-01 3.76648635e-01]
 [3.01212091e-02 8.32099497e-01 1.37779236e-01]
 [1.84562746e-02 6.85437977e-01 2.96105713e-01]
 [1.03960410e-02 5.99336743e-01 3.90267193e-01]
 [1.11942729e-02 3.03744197e-01 6.85061455e-01]
 [7.45617505e-03 2.18929693e-01 7.73614168e-01]
 [1.03300763e-02 5.79056561e-01 4.10613358e-01]
 [5.45917638e-03 9.33444381e-01 6.10964224e-02]
 [9.76395421e-03 8.56311738e-01 1.33924320e-01]
 [1.54684496e-03 2.00456586e-02 9.78407562e-01]
 [2.53536999e-02 2.07801402e-01 7.66844869e-01]
 [2.83132941e-02 2.10809391e-02 9.506057

In [14]:
predClasses = predTest.argmax(axis=-1)
print(predClasses)

[0 0 2 0 0 0 0 0 0 1 1 1 1 2 2 1 1 1 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 0 2 1 1
 2 2 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 2 0 0 0 0 2 2 0 1 1 0 0 0 0 1 1
 0 2 2 2 2 1 0 2 2 0 2 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 2 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 2 2 2 2 1 1 1 1 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2
 1 2 2 1 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 1 2 2 2]


### Test용 payload json 생성

In [20]:
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import cv2
import json
import requests
from tqdm import tqdm




#get some sample data
sample_test_data = dataTest[2]  #[0:2]
sample_test_labels = labelsTest[2] #[0:2]

print(sample_test_data.shape)

class_names = ['covid', 'normal', 'pneumonia_bac']

# pre-process data 
#sample_test_data_processed = np.expand_dims(sample_test_data / 255., axis=3)

sample_test_data_processed = sample_test_data

# create payload
data = json.dumps({"signature_name": "serving_default", 
                   "instances": sample_test_data_processed.tolist()})

(256, 256, 3)


In [21]:
with open("image_data.json", "w") as json_file:
    json.dump(data, json_file)
# print(data)

### KFServing 테스트

In [ ]:
HEADERS = {'content-type': 'application/json'}
#MODEL1_API_URL = 'http://172.17.0.1:8511/v1/models/covid19/versions/2:predict'
MODEL1_API_URL = 'http://35.206.247.237:32380/v1/models/covid19:predict'

# inference request
json_response = requests.post(MODEL1_API_URL, data=data, headers=HEADERS)

print(json_response)

pred = json.loads(json_response.content.decode('utf-8'))
print(pred)

# view server response
predictions = json.loads(json_response.text)['predictions']
predictions = np.argmax(np.array(predictions), axis=1)
prediction_labels = [class_names[p] for p in predictions]

fig, ax = plt.subplots(6, 5, figsize=(60, 80))
for idx, img in enumerate(sample_test_data):
    rowidx = idx // 5
    colidx = idx % 5
    ax[rowidx, colidx].imshow(img)
    ax[rowidx, colidx].set_title('Actual: {}\nPredicted: {}'.format(class_names[sample_test_labels[idx].argmax(axis=-1)], 
                                                                    prediction_labels[idx]), fontsize=40)